# Predicción en base a hora, latitud y longitud

In [1]:
import joblib
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv("carpetas.csv")

In [3]:
df.head()

,id,inicio,delito,categoría,colonia,alcaldía,latitud,longitud,datetime,gravedad
0,1,2016-01-01,lesiones culposas por transito vehicular en co...,delito de bajo impacto,jardines en la montaña,tlalpan,19.30086,-99.20877,2015-12-31 16:30:00,bajo impacto
1,2,2016-01-01,robo a pasajero a bordo de taxi con violencia,robo a pasajero a bordo de taxi con violencia,lomas de padierna,tlalpan,19.29003,-99.21748,2015-12-31 22:40:00,impacto medio
2,3,2016-01-01,robo a transeunte en via publica con violencia,robo a transeunte en vía pública con y sin vio...,san antonio culhuacán,iztapalapa,19.34080,-99.11431,2016-01-01 00:20:00,impacto medio
3,4,2016-01-01,robo de vehiculo de servicio particular sin vi...,robo de vehículo con y sin violencia,san juan de aragón ii sección,gustavo a. madero,19.45106,-99.08669,2015-12-31 22:00:00,impacto medio
4,5,2016-01-01,homicidios intencionales (otros),homicidio doloso,nativitas,benito juarez,19.38095,-99.13982,2015-12-31 22:30:00,alto impacto


In [ ]:
df["datetime"] = pd.to_datetime(df["datetime"])
df["hora"] = df["datetime"].dt.hour

In [ ]:
X = df[['hora', 'latitud', 'longitud']]
y = pd.DataFrame({
    'bajo_impacto': (df['gravedad'] == 'bajo impacto').astype(int),
    'impacto_medio': (df['gravedad'] == 'impacto medio').astype(int),
    'alto_impacto': (df['gravedad'] == 'alto impacto').astype(int)
})

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalizar las características
    ('modelo', MultiOutputClassifier(LogisticRegression()))  # Usar un clasificador multinomial
])

In [9]:
pipeline.fit(X_train, y_train)

TypeError: float() argument must be a string or a real number, not 'datetime.date'

In [24]:
y_pred_prob = pipeline.predict_proba(X_test)

In [25]:
prob_bajo = y_pred_prob[0][:, 1]
prob_medio = y_pred_prob[1][:, 1]
prob_alto = y_pred_prob[2][:, 1]

In [26]:
roc_bajo = roc_auc_score(y_test['bajo_impacto'], prob_bajo)
roc_medio = roc_auc_score(y_test['impacto_medio'], prob_medio)
roc_alto = roc_auc_score(y_test['alto_impacto'], prob_alto)

In [27]:
print(f'AUC para bajo impacto: {roc_bajo:.4f}')
print(f'AUC para impacto medio: {roc_medio:.4f}')
print(f'AUC para alto impacto: {roc_alto:.4f}')

AUC para bajo impacto: 0.5673
AUC para impacto medio: 0.5697
AUC para alto impacto: 0.5519


In [28]:
pipeline.predict(X_test.loc[[0]])

array([[1, 0, 0]])

In [29]:
pipeline.predict_proba(X_test.loc[[0]])

[array([[0.11736635, 0.88263365]]),
 array([[0.89436413, 0.10563587]]),
 array([[0.98835542, 0.01164458]])]

In [30]:
os.makedirs('modelos', exist_ok=True)
joblib.dump(pipeline, 'modelos/modelo_coordenadas.joblib')

['modelos/modelo_coordenadas.joblib']